In [2]:
#llm = LLM(model="EleutherAI/gpt-j-6b")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
from vllm import LLM, SamplingParams

prompts = [
    "Make a sentence using the verb 'retire'.  Sentence: ",
    "Make a sentence using the verb 'retire'.  Sentence: ",
    "Make a sentence using the verb 'retire'.  Sentence: ",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens = 40)

In [35]:
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 3/3 [00:00<00:00,  5.03it/s, est. speed input: 75.41 toks/s, output: 201.08 toks/s]

Prompt: "Make a sentence using the verb 'retire'.  Sentence: ", Generated text: "\n\nI like to retire in the afternoon.\n\nYou'll notice you've used the verb 'retire' in the sentence.  The verb 'retire' is a transitive verb"
Prompt: "Make a sentence using the verb 'retire'.  Sentence: ", Generated text: '\n\nRetirement is ____, but it can be a pleasant change.\n\nA:\n\nThe word you are looking for is abdicate. \nIt has the meaning of'
Prompt: "Make a sentence using the verb 'retire'.  Sentence: ", Generated text: '\n\nBy the time you retire, you will be 60 years old.\n\nWhat I made: \n\nBy the time you retire, you will be 60 years old.\n\nA'


### Try out Llama-8b

In [6]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
PAD_TOKEN = "<|pad|>"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})
tokenizer.padding_side = "right"
# takes 2-3 minutes
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    #     attn_implementation="flash_attention_2",
    #     attn_implementation="sdpa",
    device_map="auto",
)
model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=8)

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.98s/it]


Embedding(128264, 4096)

In [8]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    return_full_text=False,
)

In [13]:
# template
def create_test_prompt(word: str):
    prompt_format = "make three sentences in Japanese using the verb {}."
    prompt = prompt_format.format(word)
    messages = [
        {"role": "user", "content": prompt},
    ]
    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

In [16]:
# let's run some inference
# 1. generation task

# system warmup
SYSTEM_START = "You can speak both japanese and english fluently. Use Hiragana and kanji."
USER_START = "make three sentences in Japanese using the verb '慣れる'"
warmup_prompt = tokenizer.apply_chat_template(
        [{
            "role": "system",
            "content": SYSTEM_START,
        }, {
            "role": "user",
            "content": USER_START,
        }
        ], tokenize=False, add_generation_prompt=True )
outputs = pipe(warmup_prompt)

# start
WORDS_LIST = ["𠮟る","変える","行なう", "冷める"]
prompts = [create_test_prompt(word) for word in WORDS_LIST]
for prompt in prompts:
    print(prompt)
    print("\nPrediction:")
    outputs = pipe(prompt)
    print(outputs[0]["generated_text"])
    print("\n\n")

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

make three sentences in Japanese using the verb 𠮟る.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Prediction:
Here are three sentences in Japanese using the verb 𠮟る (miru, meaning "to see" or "to look"):

1. (Watashi wa ki no ji o miru.) - I look at the tree's branches.
2. (Kare wa sono inu o miru.) - He looks at that dog.
3. (Watashi wa asa made no sore o miru.) - I look at the sunrise until morning.

Note: The verb 𠮟る (miru) can have different nuances depending on the context and the particles used. In these sentences, I used the



<|begin_of_text|><|start_header_id|>user<|end_header_id|>

make three sentences in Japanese using the verb 変える.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Prediction:
Here are three sentences in Japanese using the verb 変える (kaeru), which means "to change" or "to exchange":

1. (Watashi wa kimono o kaeru.) - I will change my kimono.
2. (Kono inu wa neko o kaeru.) - This d

That was bullshit. Let's try a model that can speak Japanese.
sadly, there's only 70b model for llama 3 in japanese.
### Try out Llama2-Japanese-7b

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"
text = "クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。"
# will take 4-6 minutes
model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    return_full_text=False,
)


In [2]:
from prompt.Llama2JP_prompter import Llama2JPPrompter

WORDS_LIST = ["起きる","変える","行なう", "冷める", "謝る", "抱く", "決める", "高める"]

prompter = Llama2JPPrompter(tokenizer)
prompts = [prompter.prompt_generate_sentences_fewshot(word) for word in WORDS_LIST]

for prompt in prompts:
    print(prompt)
    print("\n\n\n")

<<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

<s>[INST] make three sentences in Japanese using the word 慣れる. [/INST]  1. 新しい環境に慣れるのには時間がかかります。
2. 毎日練習して、次第に英語を話すことに慣れてきました。
3. 韓国での生活にもすぐに慣れました。 </s><s>[INST] make three sentences in Japanese using the word 起きる. [/INST] 




<<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

<s>[INST] make three sentences in Japanese using the word 慣れる. [/INST]  1. 新しい環境に慣れるのには時間がかかります。
2. 毎日練習して、次第に英語を話すことに慣れてきました。
3. 韓国での生活にもすぐに慣れました。 </s><s>[INST] make three sentences in Japanese using the word 変える. [/INST] 




<<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

<s>[INST] make three sentences in Japanese using the word 慣れる. [/INST]  1. 新しい環境に慣れるのには時間がかかります。
2. 毎日練習して、次第に英語を話すことに慣れてきました。
3. 韓国での生活にもすぐに慣れました。 </s><s>[INST] make three sentences in Japanese using the word 行なう. [/INST] 




<<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

<s>[INST] make three sentences in Japanese using the word 慣れる. [/INST]  1. 新しい環境に慣れるのには時間がかかります。
2. 毎日練習して、次第に英語を話すことに慣れてきました。
3. 韓国での生活にもすぐに慣れました。 

In [17]:
output = pipe(prompts, batch_size = len(WORDS_LIST))

TypeError: list indices must be integers or slices, not str

In [18]:
for prompt in prompts:
    print(prompt)
    print("\nPrediction:")
    outputs = pipe(prompt)
    print(outputs[0]["generated_text"])
    print("\n\n")

<<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

<s>[INST] make three sentences in Japanese using the word 慣れる. [/INST] 1. 新しい環境に慣れるのには時間がかかります。
2. 毎日練習して、次第に英語を話すことに慣れてきました。
3. 韓国での生活にもすぐに慣れました。 </s><s>[INST] make three sentences in Japanese using the word 起きる. [/INST]

Prediction:
  1. 今日は早く起きるぞ！
2. 早く起きて海に行きましょう。
3. 彼女は早く起きて朝ごはんを作ります。



<<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

<s>[INST] make three sentences in Japanese using the word 慣れる. [/INST] 1. 新しい環境に慣れるのには時間がかかります。
2. 毎日練習して、次第に英語を話すことに慣れてきました。
3. 韓国での生活にもすぐに慣れました。 </s><s>[INST] make three sentences in Japanese using the word 変える. [/INST]

Prediction:
  1. 彼女には変えられないほど好きな人がいます。
2. 勉強のやり方を変えると結果が出ました。
3. 季節の変わり目は肌が変えられません。



<<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

<s>[INST] make three sentences in Japanese using the word 慣れる. [/INST] 1. 新しい環境に慣れるのには時間がかかります。
2. 毎日練習して、次第に英語を話すことに慣れてきました。
3. 韓国での生活にもすぐに慣れました。 </s><s>[INST] make three sentences in Japanese using the word 行なう. [/INST]

Prediction:
  1. 彼は毎日早起きして、構想を練るために筆を取り、新しい演説を作

In [19]:

SYSTEM_START = "あなたは誠実で優秀な日本人のアシスタントです。"
warmup_prompt = "{bos_token}{b_inst} {system}{e_inst} ".format(
    bos_token=tokenizer.bos_token,
    b_inst=B_INST,
    system=f"{B_SYS}{SYSTEM_START}{E_SYS}",
    e_inst=E_INST,
)
print(warmup_prompt)

def create_test_prompt(word: str):
    prompt_format = "{bos_token}{b_inst}make three different sentences in Japanese using the word '{word}'. {e_inst} "
    return prompt_format.format(
        bos_token=tokenizer.bos_token,
        b_inst=B_INST,
        word=word,
        e_inst=E_INST,
    )


prompts = [create_test_prompt(word) for word in WORDS_LIST]
for prompt in prompts:
    print(prompt)

<s>[INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

[/INST] 
<s>[INST]make three different sentences in Japanese using the word '起きる'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '変える'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '行なう'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '冷める'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '謝る'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '抱く'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '決める'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '高める'. [/INST] 


In [20]:

system_output = pipe(warmup_prompt)
for prompt in prompts:
    print(prompt)
    print("\nPrediction:")
    outputs = pipe(prompt)
    print(outputs[0]["generated_text"])
    print("\n\n")


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


<s>[INST]make three different sentences in Japanese using the word '起きる'. [/INST] 

Prediction:
 承知しました。
「起きる」を使った3つの日本語の例を示します。

1. 明日、早起きしていい気分になりそうだ。
2. 毎朝、早起きすると気持ちが良いです。
3. 私は毎日、早起きするので時間に余裕があります。

これら



<s>[INST]make three different sentences in Japanese using the word '変える'. [/INST] 

Prediction:
 承知しました。

1. 彼女は髪の毛を染めて、変えることにした。
2. 私はこの前のテストで悪い点数を取ったので、変えるぞ。
3. 彼は仕事を辞めて、変えることにした。



<s>[INST]make three different sentences in Japanese using the word '行なう'. [/INST] 

Prediction:
 承知しました。
「行なう」を使った3つの例文を提案します。

1. 彼は毎日早起きして、散歩を行なう。
2. 彼女は毎日ピアノを行なう。
3. 彼は毎日ジムに通って、体を行なう。



<s>[INST]make three different sentences in Japanese using the word '冷める'. [/INST] 

Prediction:
 承知しました。

1. コーヒーが冷めると美味しくない。
2. 彼女の態度が冷めた。
3. 夏の日の冷たいビールがおいしい。



<s>[INST]make three different sentences in Japanese using the word '謝る'. [/INST] 

Prediction:
 承知しました。

1. 私は遅くなってしまったが、来れたので謝る。
2. 友達に髪の毛切ったりして、申し訳なかった。謝る。
3. 急な仕事が入ってしまい、残念ながら来れない。謝る。



<s>[INST]make three different sentences in Japanese using the 

In [9]:
# measure inference time
import time

system_output = pipe(warmup_prompt)
duration = 0.0
NUM_WARMUP = 0
NUM_REPEATS = 1
for i in range(NUM_REPEATS+NUM_WARMUP):
    start_time = time.time()
    for prompt in prompts:
        outputs = pipe(prompt)
    finish_time = time.time()
    if i >= NUM_WARMUP:
        duration += finish_time - start_time
print(len(outputs))
print(outputs[0])
print(duration/NUM_REPEATS)

1
{'generated_text': ' 承知しました。\n「高める」を使った3つの例文を提案します。\n\n1. 彼女の歌唱力は日々高められている。\n2. この薬は胃の壁を高めて胃酸をneutralizeする。\n3. 彼の野球の技術は高められたと評判だ。\n\nこれらはあく'}
19.125016450881958


In [8]:
system_output = pipe(warmup_prompt)
duration = 0.0
for i in range(NUM_REPEATS+NUM_WARMUP):
    start_time = time.time()
    outputs = pipe(prompts, batch_size = len(prompts))
    finish_time = time.time()
    if i >= NUM_WARMUP:
        duration += finish_time - start_time
print(len(outputs))
print(outputs[0])
print(duration/NUM_REPEATS)

8
[{'generated_text': 'なんとなく、「起きる」という単語の意味を想像できていると思います。\nそれでは、今回は「起きる」という単語を使った、3つの異なる意味の例を挙げます。\n\n1. 今日は早く起きる必要がある。\n2. 彼女は毎日早く起きる。\n'}]
10.832334280014038


In [12]:
model.generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "use_cache": false
}

In [24]:
# let's look at the tokens more closely.
with torch.no_grad():
    token_ids = tokenizer.encode(prompts[4], add_special_tokens=False, return_tensors="pt")
    print(prompts[4])
    generation = model.generate(
        token_ids.to(model.device),
        max_new_tokens=256,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        #top_k=3,
        #temperature=0.4,
        num_beams = 4,
        early_stopping=True,
        num_return_sequences = 3
    )

for output_ids in generation:
    # individual tokens are just one kanji/hiragana character. not vocabulary
    output = tokenizer.decode(output_ids[token_ids.size(1) :], skip_special_tokens=True)
    print(output)

<<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

<s>[INST] make three sentences in Japanese using the word 慣れる. [/INST]  1. 新しい環境に慣れるのには時間がかかります。
2. 毎日練習して、次第に英語を話すことに慣れてきました。
3. 韓国での生活にもすぐに慣れました。 </s><s>[INST] make three sentences in Japanese using the word 謝る. [/INST] 


承知しました。「謝る」を使った3つの日本語の文章は以下の通りです。

1. 友人に迷惑をかけて謝る。
2. 遅くなったので謝る。
3. ミスをして上司に謝る。


In [51]:
import MeCab
from collections import namedtuple
tagger = MeCab.Tagger("--node-format=%f%m")
tagged = tagger.parse("彼女は毎日のように濯物洗を行なう").split('\n')

TaggedWord = namedtuple('TaggedWord', 'word, yomi, pos')

parts = []
for line in tagged:
    tags = line.split('\t')
    if len(tags) >= 5:
        parts.append(TaggedWord(word=tags[0], yomi=tags[1], pos=tags[4]))
    
print(parts)


[TaggedWord(word='彼女', yomi='カノジョ', pos='代名詞'), TaggedWord(word='は', yomi='ワ', pos='助詞-係助詞'), TaggedWord(word='毎日', yomi='マイニチ', pos='名詞-普通名詞-副詞可能'), TaggedWord(word='の', yomi='ノ', pos='助詞-格助詞'), TaggedWord(word='よう', yomi='ヨー', pos='形状詞-助動詞語幹'), TaggedWord(word='に', yomi='ニ', pos='助動詞'), TaggedWord(word='洗濯', yomi='センタク', pos='名詞-普通名詞-サ変可能'), TaggedWord(word='物', yomi='ブツ', pos='接尾辞-名詞的-一般'), TaggedWord(word='を', yomi='オ', pos='助詞-格助詞'), TaggedWord(word='行なう', yomi='オコナウ', pos='動詞-一般')]


In [34]:
import re

sents = re.split("\n[0-9][. ]*", "works out as follows\n1.  fast.\n2 slow 3. without newline!\n\n4. with new\n\n")
sents = [sent.strip() for sent in sents[1:]]
print(sents)

['fast.', 'slow 3. without newline!', 'with new']
